# Importing the packages and data

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
from sklearn.metrics import r2_score, mean_squared_error

from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

In [3]:
import sys
sys.path.insert(1, '../sar_dirichlet')
import dirichlet_regression

In [4]:
from func_test import cos_similarity, create_features_matrices, rmse_aitchison

In [5]:
import pickle

# With two features

In [6]:
n_features = 2
n_classes = 3

In [7]:
np.random.seed(21)

beta = np.array([[0.  , -1.6 , 1.],
                 [0.  , 1.8, -1.4],
                 [0.  , 1.4, -1.1 ]])

gamma_var = np.round(np.random.normal(size=(n_features)),2)

In [8]:
n_repeat = 100
list_n_samples = [50,200,1000]

In [9]:
cov_matrix = np.array([[1., 0.2], [0.2, 1.]])

In [10]:
#n_repeat = 2
#list_n_samples = [50,200]

# Estimation of the parameters

In [11]:
beta = np.array([[0.  , -.2 , .1],
                 [0.  , 2., -1.5],
                 [0.  , .5, -2. ]])

In [17]:
n_samples=1000
rho=0.9

X,Z,W = create_features_matrices(n_samples,n_features,choice_W='random_distance',nneighbors=10,cov_mat=cov_matrix)
Z[:,0] = 1

list_n_i = np.random.randint(100, high=10000, size=n_samples)
M = np.identity(n_samples) - rho*W
mu = dirichlet_regression.compute_mu_spatial(X, beta, M)

Y = np.array([np.random.multinomial(list_n_i[i],mu[i])/list_n_i[i] for i in range(n_samples)])
Y = (Y*(n_samples-1)+1/n_classes)/n_samples

In [18]:
reg_test = dirichlet_regression.dirichletRegressor(spatial=False)
reg_test.fit(X, Y, parametrization='alternative', Z=Z, fit_intercept=False, verbose=0)

In [19]:
%%time
reg_spatial = dirichlet_regression.dirichletRegressor(spatial=True, maxfun=5000)
reg_spatial.fit(X, Y, parametrization='alternative', Z=Z, W=W, fit_intercept=False, verbose=1)


CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
Wall time: 20.5 s


In [170]:
%%time
reg_spatial_ce = dirichlet_regression.dirichletRegressor(spatial=True)
reg_spatial_ce.fit(X, Y, parametrization='alternative', Z=Z, W=W, fit_intercept=False, verbose=1, loss='crossentropy')


Optimization terminated successfully.
Wall time: 171 ms


Test on another set of data (rerun the generation of X and W beforehand)

In [172]:
mu_pred = reg_spatial.pred(X,W)
print('R2:',r2_score(Y,mu_pred))
print('Cos similarity:',cos_similarity(Y, mu_pred))
print('RMSE:',mean_squared_error(Y,mu_pred,squared=False))

R2: 0.8692609927284156
Cos similarity: 0.9733589640819008
RMSE: 0.11975131184026193


In [173]:
mu_pred = reg_spatial_ce.pred(X,W)
print('R2:',r2_score(Y,mu_pred))
print('Cos similarity:',cos_similarity(Y, mu_pred))
print('RMSE:',mean_squared_error(Y,mu_pred,squared=False))

R2: 0.9988070368737851
Cos similarity: 0.9997439485404536
RMSE: 0.011072342461355118


---

In [42]:
seed=0

i=0

n_samples = list_n_samples[i]

true_params = np.concatenate([beta.flatten(),gamma_var, [rho]])

solutions_spatial_temp, solutions_no_spatial_temp = [], []
solutions_ce_spatial_temp, solutions_ce_no_spatial_temp = [], []

list_n_i = np.random.randint(100, high=10000, size=n_samples)

for _ in range(10):
    np.random.seed(seed)

    X,Z,W = create_features_matrices(n_samples,n_features,choice_W='random_distance',nneighbors=10,cov_mat=cov_matrix)
    Z[:,0] = 1
    M = np.identity(n_samples) - rho*W

    #list_n_i = np.random.randint(100, high=10000, size=n_samples)
    M = np.identity(n_samples) - rho*W
    mu = dirichlet_regression.compute_mu_spatial(X, beta, M)

    try:
        #print(np.random.multinomial(list_n_i[i],mu[i])/list_n_i[i])
        Y = np.array([np.random.multinomial(list_n_i[i],mu[i])/list_n_i[i] for i in range(n_samples)])
        Y = (Y*(n_samples-1)+1/n_classes)/n_samples
        print(Y[i])

    except RuntimeError:
        print("Factor is exactly singular")
    except np.linalg.LinAlgError:
        print("Singular matrix")

    seed+=1

[0.01580756 0.00666667 0.97752577]
[0.01869416 0.00666667 0.97463918]
[0.36845361 0.19189003 0.43965636]
[0.01821306 0.00666667 0.97512027]
[0.19862543 0.03601375 0.76536082]
[0.14185567 0.01725086 0.84089347]
[0.29580756 0.08027491 0.62391753]
[0.01869416 0.00666667 0.97463918]
[0.21113402 0.03986254 0.74900344]
[0.2438488  0.05381443 0.70233677]


In [175]:
def estimating_parameters(rho, n_repeat=100, list_n_samples=[50,200,1000], cov_matrix=None):
    list_solutions_spatial, list_solutions_no_spatial = [], []
    list_solutions_ce_spatial, list_solutions_ce_no_spatial = [], []

    seed=0

    for i in range(len(list_n_samples)):
        n_samples = list_n_samples[i]

        true_params = np.concatenate([beta.flatten(),gamma_var, [rho]])

        solutions_spatial_temp, solutions_no_spatial_temp = [], []
        solutions_ce_spatial_temp, solutions_ce_no_spatial_temp = [], []
        
        list_n_i = np.random.randint(100, high=10000, size=n_samples)
        
        for _ in range(n_repeat):
            np.random.seed(seed)

            X,Z,W = create_features_matrices(n_samples,n_features,choice_W='random_distance',nneighbors=10,cov_mat=cov_matrix)
            Z[:,0] = 1
            M = np.identity(n_samples) - rho*W
            
            #list_n_i = np.random.randint(100, high=10000, size=n_samples)
            M = np.identity(n_samples) - rho*W
            mu = dirichlet_regression.compute_mu_spatial(X, beta, M)
            
            try:
                Y = np.array([np.random.multinomial(list_n_i[i],mu[i])/list_n_i[i] for i in range(n_samples)])
                Y = (Y*(n_samples-1)+1/n_classes)/n_samples

                reg_spatial = dirichlet_regression.dirichletRegressor(spatial=True, maxfun=5000)
                reg_spatial.fit(X, Y, parametrization='alternative', Z=Z, W=W, fit_intercept=False, verbose=0)
                solutions_spatial_temp.append(np.concatenate([reg_spatial.beta.flatten(),reg_spatial.gamma,[reg_spatial.rho]]))

                reg_no_spatial = dirichlet_regression.dirichletRegressor(spatial=False, maxfun=5000)
                reg_no_spatial.fit(X, Y, parametrization='alternative', Z=Z, fit_intercept=False, verbose=0)
                solutions_no_spatial_temp.append(np.concatenate([reg_no_spatial.beta.flatten(),reg_no_spatial.gamma]))

                reg_spatial_ce = dirichlet_regression.dirichletRegressor(spatial=True, maxfun=5000)
                reg_spatial_ce.fit(X, Y, loss='crossentropy', W=W, fit_intercept=False, verbose=0)
                solutions_ce_spatial_temp.append(np.concatenate([reg_spatial_ce.beta.flatten(),[reg_spatial_ce.rho]]))

                reg_no_spatial_ce = dirichlet_regression.dirichletRegressor(spatial=False, maxfun=5000)
                reg_no_spatial_ce.fit(X, Y, loss='crossentropy', fit_intercept=False, verbose=0)
                solutions_ce_no_spatial_temp.append(reg_no_spatial_ce.beta.flatten())

            except RuntimeError:
                print("Factor is exactly singular")
            except np.linalg.LinAlgError:
                print("Singular matrix")

            seed+=1
        list_solutions_spatial.append(solutions_spatial_temp)
        list_solutions_no_spatial.append(solutions_no_spatial_temp)
        list_solutions_ce_spatial.append(solutions_ce_spatial_temp)
        list_solutions_ce_no_spatial.append(solutions_ce_no_spatial_temp)
    return(list_solutions_spatial, list_solutions_no_spatial, list_solutions_ce_spatial, list_solutions_ce_no_spatial)

## rho=0.1

In [176]:
%%time
list_solutions_spatial, list_solutions_no_spatial, list_solutions_ce_spatial, list_solutions_ce_no_spatial = estimating_parameters(0.1, n_repeat=100, list_n_samples=[50,200,1000], cov_matrix=cov_matrix)

Wall time: 37min 13s


In [177]:
true_params_effective = np.concatenate([beta[:,1:].flatten(),gamma_var, [rho]])
true_params_effective_no_spatial = np.concatenate([beta[:,1:].flatten(),gamma_var])

In [178]:
np.save('Data Dirichlet/multinomial_solutions_spatial_rho01_2.npy',list_solutions_spatial)
np.save('Data Dirichlet/multinomial_solutions_no_spatial_rho01_2.npy',list_solutions_no_spatial)
np.save('Data Dirichlet/multinomial_solutions_ce_spatial_rho01_2.npy',list_solutions_ce_spatial)
np.save('Data Dirichlet/multinomial_solutions_ce_no_spatial_rho01_2.npy',list_solutions_ce_no_spatial)

## rho=0.5

In [179]:
%%time
list_solutions_spatial, list_solutions_no_spatial, list_solutions_ce_spatial, list_solutions_ce_no_spatial = estimating_parameters(0.5, n_repeat=100, list_n_samples=[50,200,1000], cov_matrix=cov_matrix)

Singular matrix
Singular matrix


C:\Users\tnguyen001\AppData\Roaming\Python\Python38\site-packages\scipy\optimize\_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0


Wall time: 46min 22s


In [181]:
with open("Data Dirichlet/multinomial_solutions_spatial_rho05_2.npy", "wb") as fp:   #Pickling
    pickle.dump(list_solutions_spatial, fp)
    
with open("Data Dirichlet/multinomial_solutions_no_spatial_rho05_2.npy", "wb") as fp:   #Pickling
    pickle.dump(list_solutions_no_spatial, fp)
    
with open("Data Dirichlet/multinomial_solutions_ce_spatial_rho05_2.npy", "wb") as fp:   #Pickling
    pickle.dump(list_solutions_ce_spatial, fp)
    
with open("Data Dirichlet/multinomial_solutions_ce_no_spatial_rho05_2.npy", "wb") as fp:   #Pickling
    pickle.dump(list_solutions_ce_no_spatial, fp)

## rho = 0.9

In [182]:
%%time
list_solutions_spatial, list_solutions_no_spatial, list_solutions_ce_spatial, list_solutions_ce_no_spatial = estimating_parameters(0.9, n_repeat=100, list_n_samples=[50,200,1000], cov_matrix=cov_matrix)

Singular matrix
Singular matrix


C:\Users\tnguyen001\AppData\Roaming\Python\Python38\site-packages\scipy\optimize\_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
../sar_dirichlet\dirichlet_regression.py:467: RuntimeWarning: overflow encountered in exp
  phi = np.exp(np.matmul(Z,gamma_var))
../sar_dirichlet\dirichlet_regression.py:86: RuntimeWarning: overflow encountered in multiply
  sum_phi_mu_times_logY = np.sum( (phi_mu-1)*np.log(Y+epsilon) , axis=1 )
../sar_dirichlet\dirichlet_regression.py:87: RuntimeWarning: invalid value encountered in subtract
  sum_ll = np.sum(loggamma(phi) - sum_loggamma_phi_mu + sum_phi_mu_times_logY)


Wall time: 1h 10min 33s


In [183]:
np.save('Data Dirichlet/multinomial_solutions_spatial_rho09_2.npy',list_solutions_spatial)
np.save('Data Dirichlet/multinomial_solutions_no_spatial_rho09_2.npy',list_solutions_no_spatial)
np.save('Data Dirichlet/multinomial_solutions_ce_spatial_rho09_2.npy',list_solutions_ce_spatial)
np.save('Data Dirichlet/multinomial_solutions_ce_no_spatial_rho09_2.npy',list_solutions_ce_no_spatial)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (3,) + inhomogeneous part.

In [184]:
with open("Data Dirichlet/multinomial_solutions_spatial_rho09_2.npy", "wb") as fp:   #Pickling
    pickle.dump(list_solutions_spatial, fp)
    
with open("Data Dirichlet/multinomial_solutions_no_spatial_rho09_2.npy", "wb") as fp:   #Pickling
    pickle.dump(list_solutions_no_spatial, fp)
    
with open("Data Dirichlet/multinomial_solutions_ce_spatial_rho09_2.npy", "wb") as fp:   #Pickling
    pickle.dump(list_solutions_ce_spatial, fp)
    
with open("Data Dirichlet/multinomial_solutions_ce_no_spatial_rho09_2.npy", "wb") as fp:   #Pickling
    pickle.dump(list_solutions_ce_no_spatial, fp)

## rho=0

In [185]:
%%time
list_solutions_spatial, list_solutions_no_spatial, list_solutions_ce_spatial, list_solutions_ce_no_spatial = estimating_parameters(0., n_repeat=100, list_n_samples=[50,200,1000], cov_matrix=cov_matrix)

Wall time: 44min 34s


In [186]:
np.save('Data Dirichlet/multinomial_solutions_spatial_rho00_2.npy',list_solutions_spatial)
np.save('Data Dirichlet/multinomial_solutions_no_spatial_rho00_2.npy',list_solutions_no_spatial)
np.save('Data Dirichlet/multinomial_solutions_ce_spatial_rho00_2.npy',list_solutions_ce_spatial)
np.save('Data Dirichlet/multinomial_solutions_ce_no_spatial_rho00_2.npy',list_solutions_ce_no_spatial)

# Analysis of the results

In [12]:
from texttable import Texttable
import latextable

In [13]:
list_solutions_spatial_01 = np.load('Data Dirichlet/multinomial_solutions_spatial_rho01_2.npy')
list_solutions_no_spatial_01 = np.load('Data Dirichlet/multinomial_solutions_no_spatial_rho01_2.npy')
list_solutions_ce_spatial_01 = np.load('Data Dirichlet/multinomial_solutions_ce_spatial_rho01_2.npy')
list_solutions_ce_no_spatial_01 = np.load('Data Dirichlet/multinomial_solutions_ce_no_spatial_rho01_2.npy')

In [14]:
list_solutions_spatial_05 = np.load('Data Dirichlet/multinomial_solutions_spatial_rho05.npy', allow_pickle=True)
list_solutions_no_spatial_05 = np.load('Data Dirichlet/multinomial_solutions_no_spatial_rho05.npy', allow_pickle=True)
list_solutions_ce_spatial_05 = np.load('Data Dirichlet/multinomial_solutions_ce_spatial_rho05.npy', allow_pickle=True)
list_solutions_ce_no_spatial_05 = np.load('Data Dirichlet/multinomial_solutions_ce_no_spatial_rho05.npy', allow_pickle=True)

In [15]:
with open("Data Dirichlet/multinomial_solutions_spatial_rho05_2.npy", "rb") as fp:   # Unpickling
    list_solutions_spatial_05 = pickle.load(fp)
    
with open("Data Dirichlet/multinomial_solutions_no_spatial_rho05_2.npy", "rb") as fp:   # Unpickling
    list_solutions_no_spatial_05 = pickle.load(fp)
    
with open("Data Dirichlet/multinomial_solutions_ce_spatial_rho05_2.npy", "rb") as fp:   # Unpickling
    list_solutions_ce_spatial_05 = pickle.load(fp)
    
with open("Data Dirichlet/multinomial_solutions_ce_no_spatial_rho05_2.npy", "rb") as fp:   # Unpickling
    list_solutions_ce_no_spatial_05 = pickle.load(fp)

In [27]:
with open("Data Dirichlet/multinomial_solutions_spatial_rho09_2.npy", "rb") as fp:   # Unpickling
    list_solutions_spatial_09 = pickle.load(fp)
    
with open("Data Dirichlet/multinomial_solutions_no_spatial_rho09_2.npy", "rb") as fp:   # Unpickling
    list_solutions_no_spatial_09 = pickle.load(fp)
    
with open("Data Dirichlet/multinomial_solutions_ce_spatial_rho09_2.npy", "rb") as fp:   # Unpickling
    list_solutions_ce_spatial_09 = pickle.load(fp)
    
with open("Data Dirichlet/multinomial_solutions_ce_no_spatial_rho09_2.npy", "rb") as fp:   # Unpickling
    list_solutions_ce_no_spatial_09 = pickle.load(fp)

In [17]:
with open("Data Dirichlet/multinomial_solutions_spatial_rho09.npy", "rb") as fp:   # Unpickling
    list_solutions_spatial_09 = pickle.load(fp)
    
with open("Data Dirichlet/multinomial_solutions_no_spatial_rho09.npy", "rb") as fp:   # Unpickling
    list_solutions_no_spatial_09 = pickle.load(fp)
    
with open("Data Dirichlet/multinomial_solutions_ce_spatial_rho09.npy", "rb") as fp:   # Unpickling
    list_solutions_ce_spatial_09 = pickle.load(fp)
    
with open("Data Dirichlet/multinomial_solutions_ce_no_spatial_rho09.npy", "rb") as fp:   # Unpickling
    list_solutions_ce_no_spatial_09 = pickle.load(fp)

In [18]:
true_params_effective_01 = np.concatenate([beta[:,1:].flatten(),gamma_var, [0.1]])
true_params_effective_05 = np.concatenate([beta[:,1:].flatten(),gamma_var, [0.5]])
true_params_effective_09 = np.concatenate([beta[:,1:].flatten(),gamma_var, [0.9]])
true_params_effective_no_spatial = np.concatenate([beta[:,1:].flatten(),gamma_var])

In [193]:
np.save('Data Dirichlet/multinomial_solutions_spatial_rho00_2.npy',list_solutions_spatial)
np.save('Data Dirichlet/multinomial_solutions_no_spatial_rho00_2.npy',list_solutions_no_spatial)
np.save('Data Dirichlet/multinomial_solutions_ce_spatial_rho00_2.npy',list_solutions_ce_spatial)
np.save('Data Dirichlet/multinomial_solutions_ce_no_spatial_rho00_2.npy',list_solutions_ce_no_spatial)

In [19]:
list_solutions_spatial_00 = np.load('Data Dirichlet/multinomial_solutions_spatial_rho00_2.npy')
list_solutions_no_spatial_00 = np.load('Data Dirichlet/multinomial_solutions_no_spatial_rho00_2.npy')
list_solutions_ce_spatial_00 = np.load('Data Dirichlet/multinomial_solutions_ce_spatial_rho00_2.npy', allow_pickle=True)
list_solutions_ce_no_spatial_00 = np.load('Data Dirichlet/multinomial_solutions_ce_no_spatial_rho00_2.npy', allow_pickle=True)
true_params_effective_00 = np.concatenate([beta[:,1:].flatten(),gamma_var, [0.]])

### Results of maximum likelihood

In [198]:
table_params = Texttable()
table_params.add_row(["Parameter", "S n=50", "S n=200", "S n=1000",
                      "NS n=50", "NS n=200", "NS n=1000"])
                      
#param_names = ["$\\beta_{01}$", "", "$\\beta_{02}$", "", "$\\beta_{11}$", "", "$\\beta_{12}$", "",
#               "$\\beta_{21}$", "", "$\\beta_{22}$", "", "$\\gamma_0$", "", "$\\gamma_1$", "", "$\\rho$", ""]

param_names = ["\\multirow{2}{*}{$\\beta_{01}$}", "\\multirow{2}{*}{$\\beta_{02}$}", "\\multirow{2}{*}{$\\beta_{11}$}",
               "\\multirow{2}{*}{$\\beta_{12}$}", "\\multirow{2}{*}{$\\beta_{21}$}", "\\multirow{2}{*}{$\\beta_{22}$}",
               "\\multirow{2}{*}{$\\gamma_0$}", "\\multirow{2}{*}{$\\gamma_1$}", "\\multirow{2}{*}{$\\rho$}" ]
param_names_2 = [""]*len(param_names)

columns_1 = []
columns_2 = []

for ns in range(3):
    col_to_add = [str(np.round(np.mean(list_solutions_spatial_01[ns][:,i] - true_params_effective_01[i]),3))
                  +' ('+str(np.round(np.std(list_solutions_spatial_01[ns][:,i] - true_params_effective_01[i]),3))+')' for i in range(len(true_params_effective_01))]
    columns_1.append(col_to_add)
    col_to_add = ['['+  str(np.round(mean_squared_error(list_solutions_spatial_01[ns][:,i], [true_params_effective_01[i]]*100),3)) +']' for i in range(len(true_params_effective_01))]
    columns_2.append(col_to_add)
    
for ns in range(3):
    col_to_add = [str(np.round(np.mean(list_solutions_no_spatial_01[ns][:,i] - true_params_effective_no_spatial[i]),3))
                  +' ('+str(np.round(np.std(list_solutions_no_spatial_01[ns][:,i] - true_params_effective_no_spatial[i]),3))+')' for i in range(len(true_params_effective_no_spatial))] + ["/"]
    columns_1.append(col_to_add)
    col_to_add = [' ['+  str(np.round(mean_squared_error(list_solutions_no_spatial_01[ns][:,i], [true_params_effective_no_spatial[i]]*100),3)) +']' for i in range(len(true_params_effective_no_spatial))] + ["/"]
    columns_2.append(col_to_add)
        
columns_1.insert(0,param_names)
columns_2.insert(0,param_names_2)

tr_columns_1 = np.transpose(columns_1)
tr_columns_2 = np.transpose(columns_2)
for i in range(len(tr_columns_1)):
    table_params.add_row(tr_columns_1[i])
    table_params.add_row(tr_columns_2[i])

print(table_params.draw())

+-----------+-----------+----------+----------+----------+----------+----------+
| Parameter | S n=50    | S n=200  | S n=1000 | NS n=50  | NS n=200 | NS       |
|           |           |          |          |          |          | n=1000   |
+-----------+-----------+----------+----------+----------+----------+----------+
| \multirow | 0.051     | 0.016    | 0.004    | 0.043    | 0.001    | -0.008   |
| {2}{*}{$\ | (0.032)   | (0.01)   | (0.003)  | (0.038)  | (0.019)  | (0.008)  |
| beta_{01} |           |          |          |          |          |          |
| $}        |           |          |          |          |          |          |
+-----------+-----------+----------+----------+----------+----------+----------+
|           | [0.004]   | [0.0]    | [0.0]    |  [0.003] |  [0.0]   |  [0.0]   |
+-----------+-----------+----------+----------+----------+----------+----------+
| \multirow | 0.029     | 0.01     | 0.003    | 0.047    | 0.031    | 0.021    |
| {2}{*}{$\ | (0.025)   | (0

In [199]:
print(latextable.draw_latex(table_params, caption="Difference between the estimated parameters and the true parameters ($rho=0.1$) with multinomial generated data. The results are presented as the mean of the differences on the 100 iterations, the standard deviation within parenthesis, and the standard deviation within square brackets."))

\begin{table}
	\begin{center}
		\begin{tabular}{|l|l|l|l|l|l|l|}
			\hline
			 \\
			\hline
			Parameter & S n=50 & S n=200 & S n=1000 & NS n=50 & NS n=200 & NS n=1000 \\
			\hline
			\multirow{2}{*}{$\beta_{01}$} & 0.051 (0.032) & 0.016 (0.01) & 0.004 (0.003) & 0.043 (0.038) & 0.001 (0.019) & -0.008 (0.008) \\
			\hline
			 & [0.004] & [0.0] & [0.0] &  [0.003] &  [0.0] &  [0.0] \\
			\hline
			\multirow{2}{*}{$\beta_{02}$} & 0.029 (0.025) & 0.01 (0.006) & 0.003 (0.002) & 0.047 (0.05) & 0.031 (0.022) & 0.021 (0.01) \\
			\hline
			 & [0.001] & [0.0] & [0.0] &  [0.005] &  [0.001] &  [0.001] \\
			\hline
			\multirow{2}{*}{$\beta_{11}$} & -0.136 (0.046) & -0.039 (0.009) & -0.009 (0.004) & -0.15 (0.041) & -0.053 (0.016) & -0.027 (0.008) \\
			\hline
			 & [0.021] & [0.002] & [0.0] &  [0.024] &  [0.003] &  [0.001] \\
			\hline
			\multirow{2}{*}{$\beta_{12}$} & 0.143 (0.037) & 0.04 (0.012) & 0.011 (0.004) & 0.162 (0.042) & 0.057 (0.017) & 0.033 (0.008) \\
			\hline
			 & [0.022] & [0.002] 

In [200]:
table_params = Texttable()
table_params.add_row(["Parameter", "S n=50", "S n=200", "S n=1000",
                      "NS n=50", "NS n=200", "NS n=1000"])
                      
#param_names = ["$\\beta_{01}$", "", "$\\beta_{02}$", "", "$\\beta_{11}$", "", "$\\beta_{12}$", "",
#               "$\\beta_{21}$", "", "$\\beta_{22}$", "", "$\\gamma_0$", "", "$\\gamma_1$", "", "$\\rho$", ""]

param_names = ["\\multirow{2}{*}{$\\beta_{01}$}", "\\multirow{2}{*}{$\\beta_{02}$}", "\\multirow{2}{*}{$\\beta_{11}$}",
               "\\multirow{2}{*}{$\\beta_{12}$}", "\\multirow{2}{*}{$\\beta_{21}$}", "\\multirow{2}{*}{$\\beta_{22}$}",
               "\\multirow{2}{*}{$\\gamma_0$}", "\\multirow{2}{*}{$\\gamma_1$}", "\\multirow{2}{*}{$\\rho$}" ]
param_names_2 = [""]*len(param_names)

columns_1 = []
columns_2 = []

for ns in range(3):
    col_to_add = [str(np.round(np.mean(np.array(list_solutions_spatial_05[ns])[:,i] - true_params_effective_05[i]),3))
                  +' ('+str(np.round(np.std(np.array(list_solutions_spatial_05[ns])[:,i] - true_params_effective_05[i]),3))+')' for i in range(len(true_params_effective_05))]
    columns_1.append(col_to_add)
    col_to_add = ['['+  str(np.round(mean_squared_error(np.array(list_solutions_spatial_05[ns])[:,i], [true_params_effective_05[i]]*len(list_solutions_spatial_05[ns])),3)) +']' for i in range(len(true_params_effective_05))]
    columns_2.append(col_to_add)
    
for ns in range(3):
    col_to_add = [str(np.round(np.mean(np.array(list_solutions_no_spatial_05[ns])[:,i] - true_params_effective_no_spatial[i]),3))
                  +' ('+str(np.round(np.std(np.array(list_solutions_no_spatial_05[ns])[:,i] - true_params_effective_no_spatial[i]),3))+')' for i in range(len(true_params_effective_no_spatial))] + ["/"]
    columns_1.append(col_to_add)
    col_to_add = [' ['+  str(np.round(mean_squared_error(np.array(list_solutions_no_spatial_05[ns])[:,i], [true_params_effective_no_spatial[i]]*len(list_solutions_no_spatial_05[ns])),3)) +']' for i in range(len(true_params_effective_no_spatial))] + ["/"]
    columns_2.append(col_to_add)
        
columns_1.insert(0,param_names)
columns_2.insert(0,param_names_2)

tr_columns_1 = np.transpose(columns_1)
tr_columns_2 = np.transpose(columns_2)
for i in range(len(tr_columns_1)):
    table_params.add_row(tr_columns_1[i])
    table_params.add_row(tr_columns_2[i])

print(table_params.draw())

+-----------+-----------+----------+----------+----------+----------+----------+
| Parameter | S n=50    | S n=200  | S n=1000 | NS n=50  | NS n=200 | NS       |
|           |           |          |          |          |          | n=1000   |
+-----------+-----------+----------+----------+----------+----------+----------+
| \multirow | 0.041     | 0.015    | 0.007    | 0.191    | 0.157    | 0.161    |
| {2}{*}{$\ | (0.033)   | (0.02)   | (0.019)  | (0.22)   | (0.109)  | (0.044)  |
| beta_{01} |           |          |          |          |          |          |
| $}        |           |          |          |          |          |          |
+-----------+-----------+----------+----------+----------+----------+----------+
|           | [0.003]   | [0.001]  | [0.0]    |  [0.085] |  [0.036] |  [0.028] |
+-----------+-----------+----------+----------+----------+----------+----------+
| \multirow | 0.016     | 0.015    | 0.01     | 0.227    | 0.271    | 0.241    |
| {2}{*}{$\ | (0.025)   | (0

In [201]:
print(latextable.draw_latex(table_params, caption="Difference between the estimated parameters and the true parameters ($rho=0.5$) with multinomial generated data. The results are presented as the mean of the differences on the 100 iterations, the standard deviation within parenthesis, and the standard deviation within square brackets."))

\begin{table}
	\begin{center}
		\begin{tabular}{|l|l|l|l|l|l|l|}
			\hline
			 \\
			\hline
			Parameter & S n=50 & S n=200 & S n=1000 & NS n=50 & NS n=200 & NS n=1000 \\
			\hline
			\multirow{2}{*}{$\beta_{01}$} & 0.041 (0.033) & 0.015 (0.02) & 0.007 (0.019) & 0.191 (0.22) & 0.157 (0.109) & 0.161 (0.044) \\
			\hline
			 & [0.003] & [0.001] & [0.0] &  [0.085] &  [0.036] &  [0.028] \\
			\hline
			\multirow{2}{*}{$\beta_{02}$} & 0.016 (0.025) & 0.015 (0.046) & 0.01 (0.037) & 0.227 (0.264) & 0.271 (0.121) & 0.241 (0.057) \\
			\hline
			 & [0.001] & [0.002] & [0.001] &  [0.121] &  [0.088] &  [0.061] \\
			\hline
			\multirow{2}{*}{$\beta_{11}$} & -0.151 (0.033) & -0.056 (0.077) & -0.033 (0.116) & -0.616 (0.194) & -0.62 (0.103) & -0.613 (0.054) \\
			\hline
			 & [0.024] & [0.009] & [0.015] &  [0.417] &  [0.395] &  [0.378] \\
			\hline
			\multirow{2}{*}{$\beta_{12}$} & 0.163 (0.043) & 0.051 (0.042) & 0.023 (0.065) & 0.611 (0.177) & 0.592 (0.082) & 0.595 (0.043) \\
			\hline
			 & [0.02

In [202]:
table_params = Texttable()
table_params.add_row(["Parameter", "S n=50", "S n=200", "S n=1000",
                      "NS n=50", "NS n=200", "NS n=1000"])
                      
#param_names = ["$\\beta_{01}$", "", "$\\beta_{02}$", "", "$\\beta_{11}$", "", "$\\beta_{12}$", "",
#               "$\\beta_{21}$", "", "$\\beta_{22}$", "", "$\\gamma_0$", "", "$\\gamma_1$", "", "$\\rho$", ""]

param_names = ["\\multirow{2}{*}{$\\beta_{01}$}", "\\multirow{2}{*}{$\\beta_{02}$}", "\\multirow{2}{*}{$\\beta_{11}$}",
               "\\multirow{2}{*}{$\\beta_{12}$}", "\\multirow{2}{*}{$\\beta_{21}$}", "\\multirow{2}{*}{$\\beta_{22}$}",
               "\\multirow{2}{*}{$\\gamma_0$}", "\\multirow{2}{*}{$\\gamma_1$}", "\\multirow{2}{*}{$\\rho$}" ]
param_names_2 = [""]*len(param_names)

columns_1 = []
columns_2 = []

for ns in range(3):
    col_to_add = [str(np.round(np.mean(np.array(list_solutions_spatial_09[ns])[:,i] - true_params_effective_09[i]),3))
                  +' ('+str(np.round(np.std(np.array(list_solutions_spatial_09[ns])[:,i] - true_params_effective_09[i]),3))+')' for i in range(len(true_params_effective_09))]
    columns_1.append(col_to_add)
    col_to_add = ['['+  str(np.round(mean_squared_error(np.array(list_solutions_spatial_09[ns])[:,i], [true_params_effective_09[i]]*len(list_solutions_spatial_09[ns])),3)) +']' for i in range(len(true_params_effective_09))]
    columns_2.append(col_to_add)
    
for ns in range(3):
    col_to_add = [str(np.round(np.mean(np.array(list_solutions_no_spatial_09[ns])[:,i] - true_params_effective_no_spatial[i]),3))
                  +' ('+str(np.round(np.std(np.array(list_solutions_no_spatial_09[ns])[:,i] - true_params_effective_no_spatial[i]),3))+')' for i in range(len(true_params_effective_no_spatial))] + ["/"]
    columns_1.append(col_to_add)
    col_to_add = [' ['+  str(np.round(mean_squared_error(np.array(list_solutions_no_spatial_09[ns])[:,i], [true_params_effective_no_spatial[i]]*len(list_solutions_no_spatial_09[ns])),3)) +']' for i in range(len(true_params_effective_no_spatial))] + ["/"]
    columns_2.append(col_to_add)
        
columns_1.insert(0,param_names)
columns_2.insert(0,param_names_2)

tr_columns_1 = np.transpose(columns_1)
tr_columns_2 = np.transpose(columns_2)
for i in range(len(tr_columns_1)):
    table_params.add_row(tr_columns_1[i])
    table_params.add_row(tr_columns_2[i])

print(table_params.draw())

+-----------+-----------+----------+----------+----------+----------+----------+
| Parameter | S n=50    | S n=200  | S n=1000 | NS n=50  | NS n=200 | NS       |
|           |           |          |          |          |          | n=1000   |
+-----------+-----------+----------+----------+----------+----------+----------+
| \multirow | 0.117     | 0.131    | 0.12     | 0.361    | 0.18     | 0.169    |
| {2}{*}{$\ | (0.095)   | (0.093)  | (0.085)  | (0.651)  | (0.219)  | (0.084)  |
| beta_{01} |           |          |          |          |          |          |
| $}        |           |          |          |          |          |          |
+-----------+-----------+----------+----------+----------+----------+----------+
|           | [0.023]   | [0.026]  | [0.022]  |  [0.555] |  [0.08]  |  [0.036] |
+-----------+-----------+----------+----------+----------+----------+----------+
| \multirow | 0.023     | 0.001    | -0.009   | 0.548    | 0.42     | 0.311    |
| {2}{*}{$\ | (0.379)   | (0

In [203]:
print(latextable.draw_latex(table_params, caption="Difference between the estimated parameters and the true parameters ($rho=0.9$) with multinomial generated data. The results are presented as the mean of the differences on the 100 iterations, the standard deviation within parenthesis, and the standard deviation within square brackets."))

\begin{table}
	\begin{center}
		\begin{tabular}{|l|l|l|l|l|l|l|}
			\hline
			 \\
			\hline
			Parameter & S n=50 & S n=200 & S n=1000 & NS n=50 & NS n=200 & NS n=1000 \\
			\hline
			\multirow{2}{*}{$\beta_{01}$} & 0.117 (0.095) & 0.131 (0.093) & 0.12 (0.085) & 0.361 (0.651) & 0.18 (0.219) & 0.169 (0.084) \\
			\hline
			 & [0.023] & [0.026] & [0.022] &  [0.555] &  [0.08] &  [0.036] \\
			\hline
			\multirow{2}{*}{$\beta_{02}$} & 0.023 (0.379) & 0.001 (0.056) & -0.009 (0.021) & 0.548 (0.948) & 0.42 (0.334) & 0.311 (0.146) \\
			\hline
			 & [0.144] & [0.003] & [0.001] &  [1.199] &  [0.288] &  [0.118] \\
			\hline
			\multirow{2}{*}{$\beta_{11}$} & -1.097 (0.705) & -0.938 (0.665) & -0.839 (0.591) & -1.61 (0.29) & -1.639 (0.137) & -1.606 (0.063) \\
			\hline
			 & [1.7] & [1.323] & [1.053] &  [2.677] &  [2.705] &  [2.583] \\
			\hline
			\multirow{2}{*}{$\beta_{12}$} & 0.847 (0.511) & 0.763 (0.522) & 0.714 (0.496) & 1.235 (0.23) & 1.215 (0.076) & 1.196 (0.042) \\
			\hline
			 & [0.979]

In [204]:
table_params = Texttable()
table_params.add_row(["Parameter", "S n=50", "S n=200", "S n=1000",
                      "NS n=50", "NS n=200", "NS n=1000"])

param_names = ["\\multirow{2}{*}{$\\beta_{01}$}", "\\multirow{2}{*}{$\\beta_{02}$}", "\\multirow{2}{*}{$\\beta_{11}$}",
               "\\multirow{2}{*}{$\\beta_{12}$}", "\\multirow{2}{*}{$\\beta_{21}$}", "\\multirow{2}{*}{$\\beta_{22}$}",
               "\\multirow{2}{*}{$\\gamma_0$}", "\\multirow{2}{*}{$\\gamma_1$}", "\\multirow{2}{*}{$\\rho$}" ]
param_names_2 = [""]*len(param_names)

columns_1 = []
columns_2 = []

for ns in range(3):
    col_to_add = [str(np.round(np.mean(list_solutions_spatial_00[ns][:,i] - true_params_effective_00[i]),3))
                  +' ('+str(np.round(np.std(list_solutions_spatial_00[ns][:,i] - true_params_effective_00[i]),3))+')' for i in range(len(true_params_effective_00))]
    columns_1.append(col_to_add)
    col_to_add = ['['+  str(np.round(mean_squared_error(list_solutions_spatial_00[ns][:,i], [true_params_effective_00[i]]*100),3)) +']' for i in range(len(true_params_effective_00))]
    columns_2.append(col_to_add)
    
for ns in range(3):
    col_to_add = [str(np.round(np.mean(list_solutions_no_spatial_00[ns][:,i] - true_params_effective_no_spatial[i]),3))
                  +' ('+str(np.round(np.std(list_solutions_no_spatial_00[ns][:,i] - true_params_effective_no_spatial[i]),3))+')' for i in range(len(true_params_effective_no_spatial))] + ["/"]
    columns_1.append(col_to_add)
    col_to_add = [' ['+  str(np.round(mean_squared_error(list_solutions_no_spatial_00[ns][:,i], [true_params_effective_no_spatial[i]]*100),3)) +']' for i in range(len(true_params_effective_no_spatial))] + ["/"]
    columns_2.append(col_to_add)
        
columns_1.insert(0,param_names)
columns_2.insert(0,param_names_2)

tr_columns_1 = np.transpose(columns_1)
tr_columns_2 = np.transpose(columns_2)
for i in range(len(tr_columns_1)):
    table_params.add_row(tr_columns_1[i])
    table_params.add_row(tr_columns_2[i])

print(table_params.draw())

+-----------+-----------+----------+----------+----------+----------+----------+
| Parameter | S n=50    | S n=200  | S n=1000 | NS n=50  | NS n=200 | NS       |
|           |           |          |          |          |          | n=1000   |
+-----------+-----------+----------+----------+----------+----------+----------+
| \multirow | 0.052     | 0.016    | 0.005    | 0.051    | 0.015    | 0.005    |
| {2}{*}{$\ | (0.019)   | (0.012)  | (0.002)  | (0.018)  | (0.007)  | (0.002)  |
| beta_{01} |           |          |          |          |          |          |
| $}        |           |          |          |          |          |          |
+-----------+-----------+----------+----------+----------+----------+----------+
|           | [0.003]   | [0.0]    | [0.0]    |  [0.003] |  [0.0]   |  [0.0]   |
+-----------+-----------+----------+----------+----------+----------+----------+
| \multirow | 0.031     | 0.011    | 0.003    | 0.032    | 0.011    | 0.003    |
| {2}{*}{$\ | (0.018)   | (0

### Results on cross-entropy

In [205]:
true_params_effective_01_ce = np.concatenate([beta[:,1:].flatten(), [0.1]])
true_params_effective_05_ce = np.concatenate([beta[:,1:].flatten(), [0.5]])
true_params_effective_09_ce = np.concatenate([beta[:,1:].flatten(), [0.9]])
true_params_effective_no_spatial_ce = np.concatenate([beta[:,1:].flatten()])

In [206]:
table_params = Texttable()
table_params.add_row(["Parameter", "S n=50", "S n=200", "S n=1000",
                      "NS n=50", "NS n=200", "NS n=1000"])
                      
#param_names = ["$\\beta_{01}$", "", "$\\beta_{02}$", "", "$\\beta_{11}$", "", "$\\beta_{12}$", "",
#               "$\\beta_{21}$", "", "$\\beta_{22}$", "", "$\\gamma_0$", "", "$\\gamma_1$", "", "$\\rho$", ""]

param_names = ["\\multirow{2}{*}{$\\beta_{01}$}", "\\multirow{2}{*}{$\\beta_{02}$}", "\\multirow{2}{*}{$\\beta_{11}$}",
               "\\multirow{2}{*}{$\\beta_{12}$}", "\\multirow{2}{*}{$\\beta_{21}$}", "\\multirow{2}{*}{$\\beta_{22}$}",
               "\\multirow{2}{*}{$\\rho$}" ]
param_names_2 = [""]*len(param_names)

columns_1 = []
columns_2 = []

for ns in range(3):
    col_to_add = [str(np.round(np.mean(list_solutions_ce_spatial_01[ns][:,i] - true_params_effective_01_ce[i]),3))
                  +' ('+str(np.round(np.std(list_solutions_ce_spatial_01[ns][:,i] - true_params_effective_01_ce[i]),3))+')' for i in range(len(true_params_effective_01_ce))]
    columns_1.append(col_to_add)
    col_to_add = ['['+  str(np.round(mean_squared_error(list_solutions_ce_spatial_01[ns][:,i], [true_params_effective_01_ce[i]]*100),3)) +']' for i in range(len(true_params_effective_01_ce))]
    columns_2.append(col_to_add)
    
for ns in range(3):
    col_to_add = [str(np.round(np.mean(list_solutions_ce_no_spatial_01[ns][:,i] - true_params_effective_no_spatial_ce[i]),3))
                  +' ('+str(np.round(np.std(list_solutions_ce_no_spatial_01[ns][:,i] - true_params_effective_no_spatial_ce[i]),3))+')' for i in range(len(true_params_effective_no_spatial_ce))] + ["/"]
    columns_1.append(col_to_add)
    col_to_add = [' ['+  str(np.round(mean_squared_error(list_solutions_ce_no_spatial_01[ns][:,i], [true_params_effective_no_spatial_ce[i]]*100),3)) +']' for i in range(len(true_params_effective_no_spatial_ce))] + ["/"]
    columns_2.append(col_to_add)
        
columns_1.insert(0,param_names)
columns_2.insert(0,param_names_2)

tr_columns_1 = np.transpose(columns_1)
tr_columns_2 = np.transpose(columns_2)
for i in range(len(tr_columns_1)):
    table_params.add_row(tr_columns_1[i])
    table_params.add_row(tr_columns_2[i])

print(table_params.draw())

+-----------+-----------+----------+----------+----------+----------+----------+
| Parameter | S n=50    | S n=200  | S n=1000 | NS n=50  | NS n=200 | NS       |
|           |           |          |          |          |          | n=1000   |
+-----------+-----------+----------+----------+----------+----------+----------+
| \multirow | 0.067     | 0.018    | 0.003    | 0.057    | -0.001   | -0.014   |
| {2}{*}{$\ | (0.019)   | (0.006)  | (0.002)  | (0.038)  | (0.019)  | (0.008)  |
| beta_{01} |           |          |          |          |          |          |
| $}        |           |          |          |          |          |          |
+-----------+-----------+----------+----------+----------+----------+----------+
|           | [0.005]   | [0.0]    | [0.0]    |  [0.005] |  [0.0]   |  [0.0]   |
+-----------+-----------+----------+----------+----------+----------+----------+
| \multirow | 0.043     | 0.012    | 0.003    | 0.059    | 0.03     | 0.017    |
| {2}{*}{$\ | (0.019)   | (0

In [207]:
print(latextable.draw_latex(table_params, caption="Difference between the estimated parameters with the cross-entropy minimization and the true parameters ($rho=0.1$). The results are presented as the mean of the differences on the 100 iterations, the standard deviation within parenthesis, and the standard deviation within square brackets."))

\begin{table}
	\begin{center}
		\begin{tabular}{|l|l|l|l|l|l|l|}
			\hline
			 \\
			\hline
			Parameter & S n=50 & S n=200 & S n=1000 & NS n=50 & NS n=200 & NS n=1000 \\
			\hline
			\multirow{2}{*}{$\beta_{01}$} & 0.067 (0.019) & 0.018 (0.006) & 0.003 (0.002) & 0.057 (0.038) & -0.001 (0.019) & -0.014 (0.008) \\
			\hline
			 & [0.005] & [0.0] & [0.0] &  [0.005] &  [0.0] &  [0.0] \\
			\hline
			\multirow{2}{*}{$\beta_{02}$} & 0.043 (0.019) & 0.012 (0.006) & 0.003 (0.002) & 0.059 (0.05) & 0.03 (0.022) & 0.017 (0.01) \\
			\hline
			 & [0.002] & [0.0] & [0.0] &  [0.006] &  [0.001] &  [0.0] \\
			\hline
			\multirow{2}{*}{$\beta_{11}$} & -0.159 (0.03) & -0.043 (0.008) & -0.008 (0.004) & -0.163 (0.041) & -0.043 (0.017) & -0.011 (0.007) \\
			\hline
			 & [0.026] & [0.002] & [0.0] &  [0.028] &  [0.002] &  [0.0] \\
			\hline
			\multirow{2}{*}{$\beta_{12}$} & 0.176 (0.037) & 0.046 (0.01) & 0.01 (0.004) & 0.181 (0.045) & 0.048 (0.017) & 0.017 (0.008) \\
			\hline
			 & [0.032] & [0.002] & [

In [208]:
table_params = Texttable()
table_params.add_row(["Parameter", "S n=50", "S n=200", "S n=1000",
                      "NS n=50", "NS n=200", "NS n=1000"])
                      
#param_names = ["$\\beta_{01}$", "", "$\\beta_{02}$", "", "$\\beta_{11}$", "", "$\\beta_{12}$", "",
#               "$\\beta_{21}$", "", "$\\beta_{22}$", "", "$\\gamma_0$", "", "$\\gamma_1$", "", "$\\rho$", ""]

param_names = ["\\multirow{2}{*}{$\\beta_{01}$}", "\\multirow{2}{*}{$\\beta_{02}$}", "\\multirow{2}{*}{$\\beta_{11}$}",
               "\\multirow{2}{*}{$\\beta_{12}$}", "\\multirow{2}{*}{$\\beta_{21}$}", "\\multirow{2}{*}{$\\beta_{22}$}",
               "\\multirow{2}{*}{$\\rho$}" ]
param_names_2 = [""]*len(param_names)

columns_1 = []
columns_2 = []

for ns in range(3):
    col_to_add = [str(np.round(np.mean(np.array(list_solutions_ce_spatial_05[ns])[:,i] - true_params_effective_05_ce[i]),3))
                  +' ('+str(np.round(np.std(np.array(list_solutions_ce_spatial_05[ns])[:,i] - true_params_effective_05_ce[i]),3))+')' for i in range(len(true_params_effective_05_ce))]
    columns_1.append(col_to_add)
    col_to_add = ['['+  str(np.round(mean_squared_error(np.array(list_solutions_ce_spatial_05[ns])[:,i], [true_params_effective_05_ce[i]]*len(list_solutions_ce_spatial_05[ns])),3)) +']' for i in range(len(true_params_effective_05_ce))]
    columns_2.append(col_to_add)
    
for ns in range(3):
    col_to_add = [str(np.round(np.mean(np.array(list_solutions_ce_no_spatial_05[ns])[:,i] - true_params_effective_no_spatial_ce[i]),3))
                  +' ('+str(np.round(np.std(np.array(list_solutions_ce_no_spatial_05[ns])[:,i] - true_params_effective_no_spatial_ce[i]),3))+')' for i in range(len(true_params_effective_no_spatial_ce))] + ["/"]
    columns_1.append(col_to_add)
    col_to_add = [' ['+  str(np.round(mean_squared_error(np.array(list_solutions_ce_no_spatial_05[ns])[:,i], [true_params_effective_no_spatial_ce[i]]*len(list_solutions_ce_no_spatial_05[ns])),3)) +']' for i in range(len(true_params_effective_no_spatial_ce))] + ["/"]
    columns_2.append(col_to_add)
        
columns_1.insert(0,param_names)
columns_2.insert(0,param_names_2)

tr_columns_1 = np.transpose(columns_1)
tr_columns_2 = np.transpose(columns_2)
for i in range(len(tr_columns_1)):
    table_params.add_row(tr_columns_1[i])
    table_params.add_row(tr_columns_2[i])

print(table_params.draw())

+-----------+-----------+----------+----------+----------+----------+----------+
| Parameter | S n=50    | S n=200  | S n=1000 | NS n=50  | NS n=200 | NS       |
|           |           |          |          |          |          | n=1000   |
+-----------+-----------+----------+----------+----------+----------+----------+
| \multirow | 0.052     | 0.013    | 0.003    | 0.076    | -0.005   | 0.015    |
| {2}{*}{$\ | (0.014)   | (0.004)  | (0.001)  | (0.265)  | (0.143)  | (0.058)  |
| beta_{01} |           |          |          |          |          |          |
| $}        |           |          |          |          |          |          |
+-----------+-----------+----------+----------+----------+----------+----------+
|           | [0.003]   | [0.0]    | [0.0]    |  [0.076] |  [0.02]  |  [0.004] |
+-----------+-----------+----------+----------+----------+----------+----------+
| \multirow | 0.022     | 0.007    | 0.001    | 0.219    | 0.257    | 0.232    |
| {2}{*}{$\ | (0.015)   | (0

In [209]:
print(latextable.draw_latex(table_params, caption="Difference between the estimated parameters with the cross-entropy minimization and the true parameters ($rho=0.5$). The results are presented as the mean of the differences on the 100 iterations, the standard deviation within parenthesis, and the standard deviation within square brackets."))

\begin{table}
	\begin{center}
		\begin{tabular}{|l|l|l|l|l|l|l|}
			\hline
			 \\
			\hline
			Parameter & S n=50 & S n=200 & S n=1000 & NS n=50 & NS n=200 & NS n=1000 \\
			\hline
			\multirow{2}{*}{$\beta_{01}$} & 0.052 (0.014) & 0.013 (0.004) & 0.003 (0.001) & 0.076 (0.265) & -0.005 (0.143) & 0.015 (0.058) \\
			\hline
			 & [0.003] & [0.0] & [0.0] &  [0.076] &  [0.02] &  [0.004] \\
			\hline
			\multirow{2}{*}{$\beta_{02}$} & 0.022 (0.015) & 0.007 (0.004) & 0.001 (0.001) & 0.219 (0.321) & 0.257 (0.161) & 0.232 (0.074) \\
			\hline
			 & [0.001] & [0.0] & [0.0] &  [0.151] &  [0.092] &  [0.059] \\
			\hline
			\multirow{2}{*}{$\beta_{11}$} & -0.182 (0.037) & -0.047 (0.008) & -0.009 (0.004) & -0.278 (0.142) & -0.171 (0.087) & -0.157 (0.038) \\
			\hline
			 & [0.035] & [0.002] & [0.0] &  [0.098] &  [0.037] &  [0.026] \\
			\hline
			\multirow{2}{*}{$\beta_{12}$} & 0.2 (0.049) & 0.052 (0.01) & 0.01 (0.003) & 0.34 (0.168) & 0.242 (0.081) & 0.238 (0.044) \\
			\hline
			 & [0.042] & [0.0

In [210]:
table_params = Texttable()
table_params.add_row(["Parameter", "S n=50", "S n=200", "S n=1000",
                      "NS n=50", "NS n=200", "NS n=1000"])
                      
#param_names = ["$\\beta_{01}$", "", "$\\beta_{02}$", "", "$\\beta_{11}$", "", "$\\beta_{12}$", "",
#               "$\\beta_{21}$", "", "$\\beta_{22}$", "", "$\\gamma_0$", "", "$\\gamma_1$", "", "$\\rho$", ""]

param_names = ["\\multirow{2}{*}{$\\beta_{01}$}", "\\multirow{2}{*}{$\\beta_{02}$}", "\\multirow{2}{*}{$\\beta_{11}$}",
               "\\multirow{2}{*}{$\\beta_{12}$}", "\\multirow{2}{*}{$\\beta_{21}$}", "\\multirow{2}{*}{$\\beta_{22}$}",
               "\\multirow{2}{*}{$\\rho$}" ]
param_names_2 = [""]*len(param_names)

columns_1 = []
columns_2 = []

for ns in range(3):
    col_to_add = [str(np.round(np.mean(np.array(list_solutions_ce_spatial_09[ns])[:,i] - true_params_effective_09_ce[i]),3))
                  +' ('+str(np.round(np.std(np.array(list_solutions_ce_spatial_09[ns])[:,i] - true_params_effective_09_ce[i]),3))+')' for i in range(len(true_params_effective_09_ce))]
    columns_1.append(col_to_add)
    col_to_add = ['['+  str(np.round(mean_squared_error(np.array(list_solutions_ce_spatial_09[ns])[:,i], [true_params_effective_09_ce[i]]*len(list_solutions_ce_spatial_09[ns])),3)) +']' for i in range(len(true_params_effective_09_ce))]
    columns_2.append(col_to_add)
    
for ns in range(3):
    col_to_add = [str(np.round(np.mean(np.array(list_solutions_ce_no_spatial_09[ns])[:,i] - true_params_effective_no_spatial_ce[i]),3))
                  +' ('+str(np.round(np.std(np.array(list_solutions_ce_no_spatial_09[ns])[:,i] - true_params_effective_no_spatial_ce[i]),3))+')' for i in range(len(true_params_effective_no_spatial_ce))] + ["/"]
    columns_1.append(col_to_add)
    col_to_add = [' ['+  str(np.round(mean_squared_error(np.array(list_solutions_ce_no_spatial_09[ns])[:,i], [true_params_effective_no_spatial_ce[i]]*len(list_solutions_ce_no_spatial_09[ns])),3)) +']' for i in range(len(true_params_effective_no_spatial_ce))] + ["/"]
    columns_2.append(col_to_add)
        
columns_1.insert(0,param_names)
columns_2.insert(0,param_names_2)

tr_columns_1 = np.transpose(columns_1)
tr_columns_2 = np.transpose(columns_2)
for i in range(len(tr_columns_1)):
    table_params.add_row(tr_columns_1[i])
    table_params.add_row(tr_columns_2[i])

print(table_params.draw())

+-----------+-----------+----------+----------+----------+----------+----------+
| Parameter | S n=50    | S n=200  | S n=1000 | NS n=50  | NS n=200 | NS       |
|           |           |          |          |          |          | n=1000   |
+-----------+-----------+----------+----------+----------+----------+----------+
| \multirow | 0.072     | 0.023    | 0.004    | 0.405    | 0.367    | 0.496    |
| {2}{*}{$\ | (0.053)   | (0.01)   | (0.001)  | (1.403)  | (0.698)  | (0.245)  |
| beta_{01} |           |          |          |          |          |          |
| $}        |           |          |          |          |          |          |
+-----------+-----------+----------+----------+----------+----------+----------+
|           | [0.008]   | [0.001]  | [0.0]    |  [2.132] |  [0.622] |  [0.307] |
+-----------+-----------+----------+----------+----------+----------+----------+
| \multirow | 0.006     | -0.003   | -0.001   | 0.954    | 1.113    | 1.001    |
| {2}{*}{$\ | (0.114)   | (0

In [211]:
print(latextable.draw_latex(table_params, caption="Difference between the estimated parameters with the cross-entropy minimization and the true parameters ($rho=0.9$). The results are presented as the mean of the differences on the 100 iterations, the standard deviation within parenthesis, and the standard deviation within square brackets."))

\begin{table}
	\begin{center}
		\begin{tabular}{|l|l|l|l|l|l|l|}
			\hline
			 \\
			\hline
			Parameter & S n=50 & S n=200 & S n=1000 & NS n=50 & NS n=200 & NS n=1000 \\
			\hline
			\multirow{2}{*}{$\beta_{01}$} & 0.072 (0.053) & 0.023 (0.01) & 0.004 (0.001) & 0.405 (1.403) & 0.367 (0.698) & 0.496 (0.245) \\
			\hline
			 & [0.008] & [0.001] & [0.0] &  [2.132] &  [0.622] &  [0.307] \\
			\hline
			\multirow{2}{*}{$\beta_{02}$} & 0.006 (0.114) & -0.003 (0.003) & -0.001 (0.0) & 0.954 (1.45) & 1.113 (0.652) & 1.001 (0.289) \\
			\hline
			 & [0.013] & [0.0] & [0.0] &  [3.013] &  [1.663] &  [1.087] \\
			\hline
			\multirow{2}{*}{$\beta_{11}$} & -0.637 (0.511) & -0.157 (0.067) & -0.029 (0.007) & -0.99 (0.448) & -0.99 (0.239) & -1.028 (0.081) \\
			\hline
			 & [0.666] & [0.029] & [0.001] &  [1.181] &  [1.036] &  [1.064] \\
			\hline
			\multirow{2}{*}{$\beta_{12}$} & 0.545 (0.318) & 0.161 (0.065) & 0.031 (0.007) & 0.982 (0.363) & 1.136 (0.218) & 1.176 (0.1) \\
			\hline
			 & [0.398] & [

# Analysis of the results on a test set

In [20]:
def analysis_results(list_solutions_spatial, list_solutions_no_spatial, rho,
                     n_repeat=100, size_test=1000, list_n_samples=[50,200,1000]):
    
    list_r2_s, list_rmse_s, list_similarity_s, list_crossentropy_s = [], [], [], []
    list_r2_ns, list_rmse_ns, list_similarity_ns, list_crossentropy_ns = [], [], [], []

    seed=0

    for n_samples_index in range(3):
        n_samples = list_n_samples[n_samples_index]
        
        true_params = np.concatenate([beta.flatten(),gamma_var, [rho]])
        
        list_n_i = np.random.randint(100, high=10000, size=n_samples)
        
        list_r2_s_temp, list_rmse_s_temp, list_similarity_s_temp, list_crossentropy_s_temp = [], [], [], []
        list_r2_ns_temp, list_rmse_ns_temp, list_similarity_ns_temp, list_crossentropy_ns_temp = [], [], [], []
        
        for i in range(len(list_solutions_spatial[n_samples_index])):
            np.random.seed(seed+1000)

            X,Z,W = create_features_matrices(n_samples,n_features,choice_W='random_distance',nneighbors=10,cov_mat=cov_matrix)
            Z[:,0] = 1
            
            #list_n_i = np.random.randint(100, high=10000, size=n_samples)
            M = np.identity(n_samples) - rho*W
            mu = dirichlet_regression.compute_mu_spatial(X, beta, M)
            try:
                
                Y = np.array([np.random.multinomial(list_n_i[i],mu[i])/list_n_i[i] for i in range(n_samples)])
                Y = (Y*(n_samples-1)+1/n_classes)/n_samples

                solution_spatial = list_solutions_spatial[n_samples_index][i]
                beta_sol_s = np.zeros((n_features+1,n_classes))
                beta_sol_s[:,1:] = solution_spatial[:(n_features+1)*(n_classes-1)].reshape((n_features+1),n_classes-1)
                rho_sol_s = solution_spatial[-1]
                M_sol = np.identity(n_samples) - rho_sol_s*W
                mu_sol_s = dirichlet_regression.compute_mu_spatial(X, beta_sol_s, M_sol)

                list_r2_s_temp.append(r2_score(Y,mu_sol_s))
                list_rmse_s_temp.append(mean_squared_error(Y,mu_sol_s,squared=False))
                list_crossentropy_s_temp.append(-(1/size_test)*np.sum(Y*np.log(mu_sol_s)))
                list_similarity_s_temp.append(cos_similarity(Y, mu_sol_s))

                solution_no_spatial = list_solutions_no_spatial[n_samples_index][i]
                beta_sol_ns = np.zeros((n_features+1,n_classes))
                beta_sol_ns[:,1:] = solution_no_spatial[:(n_features+1)*(n_classes-1)].reshape((n_features+1),n_classes-1)
                mu_sol_ns = dirichlet_regression.compute_mu(X, beta_sol_ns)

                list_r2_ns_temp.append(r2_score(Y,mu_sol_ns))
                list_rmse_ns_temp.append(mean_squared_error(Y,mu_sol_ns,squared=False))
                list_crossentropy_ns_temp.append(-(1/size_test)*np.sum(Y*np.log(mu_sol_ns)))
                list_similarity_ns_temp.append(cos_similarity(Y, mu_sol_ns))

            except RuntimeError:
                print("Factor is exactly singular")
            except np.linalg.LinAlgError:
                print("Singular matrix")

            seed+=1
        list_r2_s.append(list_r2_s_temp)
        list_rmse_s.append(list_rmse_s_temp)
        list_similarity_s.append(list_similarity_s_temp)
        list_crossentropy_s.append(list_crossentropy_s_temp)
        list_r2_ns.append(list_r2_ns_temp)
        list_rmse_ns.append(list_rmse_ns_temp)
        list_similarity_ns.append(list_similarity_ns_temp)
        list_crossentropy_ns.append(list_crossentropy_ns_temp)
    return(list_r2_s, list_rmse_s, list_similarity_s, list_crossentropy_s,
           list_r2_ns, list_rmse_ns, list_similarity_ns, list_crossentropy_ns)

In [20]:
def analysis_results(list_solutions_spatial, list_solutions_no_spatial, rho,
                     n_repeat=100, size_test=1000, list_n_samples=[50,200,1000]):
    
    list_r2_s, list_rmse_s, list_similarity_s, list_crossentropy_s, list_rmse_a_s = [], [], [], [], []
    list_r2_ns, list_rmse_ns, list_similarity_ns, list_crossentropy_ns, list_rmse_a_ns = [], [], [], [], []

    seed=0

    for n_samples_index in range(3):
        n_samples = list_n_samples[n_samples_index]
        
        true_params = np.concatenate([beta.flatten(),gamma_var, [rho]])
        
        list_n_i = np.random.randint(100, high=10000, size=n_samples)
        
        list_r2_s_temp, list_rmse_s_temp, list_similarity_s_temp, list_crossentropy_s_temp, list_rmse_a_s_temp = [], [], [], [], []
        list_r2_ns_temp, list_rmse_ns_temp, list_similarity_ns_temp, list_crossentropy_ns_temp, list_rmse_a_ns_temp = [], [], [], [], []
        
        for i in range(len(list_solutions_spatial[n_samples_index])):
            np.random.seed(seed+1000)

            X,Z,W = create_features_matrices(n_samples,n_features,choice_W='random_distance',nneighbors=10,cov_mat=cov_matrix)
            Z[:,0] = 1
            
            #list_n_i = np.random.randint(100, high=10000, size=n_samples)
            M = np.identity(n_samples) - rho*W
            mu = dirichlet_regression.compute_mu_spatial(X, beta, M)
            try:
                
                Y = np.array([np.random.multinomial(list_n_i[i],mu[i])/list_n_i[i] for i in range(n_samples)])
                Y = (Y*(n_samples-1)+1/n_classes)/n_samples

                solution_spatial = list_solutions_spatial[n_samples_index][i]
                beta_sol_s = np.zeros((n_features+1,n_classes))
                beta_sol_s[:,1:] = solution_spatial[:(n_features+1)*(n_classes-1)].reshape((n_features+1),n_classes-1)
                rho_sol_s = solution_spatial[-1]
                M_sol = np.identity(n_samples) - rho_sol_s*W
                mu_sol_s = dirichlet_regression.compute_mu_spatial(X, beta_sol_s, M_sol)

                list_r2_s_temp.append(r2_score(Y,mu_sol_s))
                list_rmse_s_temp.append(mean_squared_error(Y,mu_sol_s,squared=False))
                list_crossentropy_s_temp.append(-(1/size_test)*np.sum(Y*np.log(mu_sol_s)))
                list_similarity_s_temp.append(cos_similarity(Y, mu_sol_s))
                list_rmse_a_s_temp.append(rmse_aitchison(Y,mu_sol_s))

                solution_no_spatial = list_solutions_no_spatial[n_samples_index][i]
                beta_sol_ns = np.zeros((n_features+1,n_classes))
                beta_sol_ns[:,1:] = solution_no_spatial[:(n_features+1)*(n_classes-1)].reshape((n_features+1),n_classes-1)
                mu_sol_ns = dirichlet_regression.compute_mu(X, beta_sol_ns)

                list_r2_ns_temp.append(r2_score(Y,mu_sol_ns))
                list_rmse_ns_temp.append(mean_squared_error(Y,mu_sol_ns,squared=False))
                list_crossentropy_ns_temp.append(-(1/size_test)*np.sum(Y*np.log(mu_sol_ns)))
                list_similarity_ns_temp.append(cos_similarity(Y, mu_sol_ns))
                list_rmse_a_ns_temp.append(rmse_aitchison(Y,mu_sol_ns))

            except RuntimeError:
                print("Factor is exactly singular")
            except np.linalg.LinAlgError:
                print("Singular matrix")

            seed+=1
        list_r2_s.append(list_r2_s_temp)
        list_rmse_s.append(list_rmse_s_temp)
        list_similarity_s.append(list_similarity_s_temp)
        list_crossentropy_s.append(list_crossentropy_s_temp)
        list_rmse_a_s.append(list_rmse_a_s_temp)
        
        list_r2_ns.append(list_r2_ns_temp)
        list_rmse_ns.append(list_rmse_ns_temp)
        list_similarity_ns.append(list_similarity_ns_temp)
        list_crossentropy_ns.append(list_crossentropy_ns_temp)
        list_rmse_a_ns.append(list_rmse_a_ns_temp)
        
    return(list_r2_s, list_rmse_s, list_similarity_s, list_crossentropy_s, list_rmse_a_s,
           list_r2_ns, list_rmse_ns, list_similarity_ns, list_crossentropy_ns, list_rmse_a_ns)

In [213]:
def details_r2_mu(list_solutions_spatial, list_solutions_no_spatial, rho,
                     n_repeat=100, size_test=1000, list_n_samples=[50,200,1000]):
    
    list_r2_s, list_r2_ns = [], []
    list_mu_s, list_mu_ns = [], []
    list_true_mu = []

    seed=0

    for n_samples_index in range(3):
        n_samples = list_n_samples[n_samples_index]
        
        true_params = np.concatenate([beta.flatten(),gamma_var, [rho]])
        
        list_n_i = np.random.randint(100, high=10000, size=n_samples)
        
        list_r2_s_temp, list_r2_ns_temp = [], []
        list_mu_s_temp, list_mu_ns_temp = [], []
        list_true_mu_temp = []
        
        for i in range(len(list_solutions_spatial[n_samples_index])):
            np.random.seed(seed+1000)

            X,Z,W = create_features_matrices(n_samples,n_features,choice_W='random_distance',nneighbors=10,cov_mat=cov_matrix)
            Z[:,0] = 1
            
            #list_n_i = np.random.randint(100, high=10000, size=n_samples)
            M = np.identity(n_samples) - rho*W
            mu = dirichlet_regression.compute_mu_spatial(X, beta, M)
            try:
                
                Y = np.array([np.random.multinomial(list_n_i[i],mu[i])/list_n_i[i] for i in range(n_samples)])
                Y = (Y*(n_samples-1)+1/n_classes)/n_samples
                
                list_true_mu_temp.append(Y)

                solution_spatial = list_solutions_spatial[n_samples_index][i]
                beta_sol_s = np.zeros((n_features+1,n_classes))
                beta_sol_s[:,1:] = solution_spatial[:(n_features+1)*(n_classes-1)].reshape((n_features+1),n_classes-1)
                rho_sol_s = solution_spatial[-1]
                M_sol = np.identity(n_samples) - rho_sol_s*W
                mu_sol_s = dirichlet_regression.compute_mu_spatial(X, beta_sol_s, M_sol)

                list_mu_s_temp.append(mu_sol_s)
                list_r2_s_temp.append(r2_score(Y,mu_sol_s, multioutput='raw_values'))

                solution_no_spatial = list_solutions_no_spatial[n_samples_index][i]
                beta_sol_ns = np.zeros((n_features+1,n_classes))
                beta_sol_ns[:,1:] = solution_no_spatial[:(n_features+1)*(n_classes-1)].reshape((n_features+1),n_classes-1)
                mu_sol_ns = dirichlet_regression.compute_mu(X, beta_sol_ns)

                list_mu_ns_temp.append(mu_sol_ns)
                list_r2_ns_temp.append(r2_score(Y,mu_sol_ns, multioutput='raw_values'))

            except RuntimeError:
                print("Factor is exactly singular")
            except np.linalg.LinAlgError:
                print("Singular matrix")

            seed+=1
        list_true_mu.append(list_true_mu_temp)
        list_mu_s.append(list_mu_s_temp)
        list_mu_ns.append(list_mu_ns_temp)
        list_r2_s.append(list_r2_s_temp)
        list_r2_ns.append(list_r2_ns_temp)
    return(list_r2_s, list_r2_ns, list_true_mu, list_mu_s, list_mu_ns)

## rho=0.1

In [24]:
%%time
list_r2_s, list_rmse_s, list_similarity_s, list_crossentropy_s, list_rmse_a_s, list_r2_ns, list_rmse_ns, list_similarity_ns, list_crossentropy_ns, list_rmse_a_ns = analysis_results(list_solutions_spatial_01, list_solutions_no_spatial_01, rho=0.1) 

Wall time: 9.85 s


In [26]:
print("DIRICHLET no spatial")
print('R2', np.round(np.mean(list_r2_ns[2]),4), '(', np.round(np.var(list_r2_ns[2]),4), ')')
print('RMSE', np.round(np.mean(list_rmse_ns[2]),4), '(', np.round(np.var(list_rmse_ns[2]),4), ')')
print('Crossentropy', np.round(np.mean(list_crossentropy_ns[2]),4), '(', np.round(np.var(list_crossentropy_ns[2]),4), ')')
print('Cos similarity', np.round(np.mean(list_similarity_ns[2]),4), '(', np.round(np.var(list_similarity_ns[2]),4), ')')
print('RMSE_A', np.round(np.mean(list_rmse_a_ns[2]),4), '(', np.round(np.var(list_rmse_a_ns[2]),4), ')')
print("---\nDIRICHLET spatial")
print('R2', np.round(np.mean(list_r2_s[2]),4), '(', np.round(np.var(list_r2_s[2]),4), ')')
print('RMSE', np.round(np.mean(list_rmse_s[2]),4), '(', np.round(np.var(list_rmse_s[2]),4), ')')
print('Crossentropy', np.round(np.mean(list_crossentropy_s[2]),4), '(', np.round(np.var(list_crossentropy_s[2]),4), ')')
print('Cos similarity', np.round(np.mean(list_similarity_s[2]),4), '(', np.round(np.var(list_similarity_s[2]),4), ')')
print('RMSE_A', np.round(np.mean(list_rmse_a_s[2]),4), '(', np.round(np.var(list_rmse_a_s[2]),4), ')')

DIRICHLET no spatial
R2 0.9955 ( 0.0 )
RMSE 0.018 ( 0.0 )
Crossentropy 0.588 ( 0.0001 )
Cos similarity 0.9991 ( 0.0 )
RMSE_A 0.3133 ( 0.0004 )
---
DIRICHLET spatial
R2 0.999 ( 0.0 )
RMSE 0.0071 ( 0.0 )
Crossentropy 0.5866 ( 0.0001 )
Cos similarity 0.9999 ( 0.0 )
RMSE_A 0.2643 ( 0.0004 )


In [41]:
%%time
list_r2_s, list_rmse_s, list_similarity_s, list_crossentropy_s, list_rmse_a_s, list_r2_ns, list_rmse_ns, list_similarity_ns, list_crossentropy_ns, list_rmse_a_ns = analysis_results(list_solutions_ce_spatial_01, list_solutions_ce_no_spatial_01, rho=0.1)

Wall time: 22.3 s


In [42]:
print("CROSSENTROPY no spatial")
print('R2', np.round(np.mean(list_r2_ns[2]),4), '(', np.round(np.var(list_r2_ns[2]),4), ')')
print('RMSE', np.round(np.mean(list_rmse_ns[2]),4), '(', np.round(np.var(list_rmse_ns[2]),4), ')')
print('Crossentropy', np.round(np.mean(list_crossentropy_ns[2]),4), '(', np.round(np.var(list_crossentropy_ns[2]),4), ')')
print('Cos similarity', np.round(np.mean(list_similarity_ns[2]),4), '(', np.round(np.var(list_similarity_ns[2]),4), ')')
print('RMSE_A', np.round(np.mean(list_rmse_a_ns[2]),4), '(', np.round(np.var(list_rmse_a_ns[2]),4), ')')

print("---\nCROSSENTROPY spatial")
print('R2', np.round(np.mean(list_r2_s[2]),4), '(', np.round(np.var(list_r2_s[2]),4), ')')
print('RMSE', np.round(np.mean(list_rmse_s[2]),4), '(', np.round(np.var(list_rmse_s[2]),4), ')')
print('Crossentropy', np.round(np.mean(list_crossentropy_s[2]),4), '(', np.round(np.var(list_crossentropy_s[2]),4), ')')
print('Cos similarity', np.round(np.mean(list_similarity_s[2]),4), '(', np.round(np.var(list_similarity_s[2]),4), ')')
print('RMSE_A', np.round(np.mean(list_rmse_a_s[2]),4), '(', np.round(np.var(list_rmse_a_s[2]),4), ')')

CROSSENTROPY no spatial
R2 0.9956 ( 0.0 )
RMSE 0.0179 ( 0.0 )
Crossentropy 0.5881 ( 0.0001 )
Cos similarity 0.9991 ( 0.0 )
RMSE_A 0.3184 ( 0.0004 )
---
CROSSENTROPY spatial
R2 0.999 ( 0.0 )
RMSE 0.007 ( 0.0 )
Crossentropy 0.5866 ( 0.0001 )
Cos similarity 0.9999 ( 0.0 )
RMSE_A 0.2648 ( 0.0003 )


In [214]:
%%time
list_r2_s, list_rmse_s, list_similarity_s, list_crossentropy_s, list_r2_ns, list_rmse_ns, list_similarity_ns, list_crossentropy_ns = analysis_results(list_solutions_spatial_01, list_solutions_no_spatial_01, rho=0.1) 

Wall time: 17.5 s


In [215]:
print("DIRICHLET no spatial")
print('R2', np.round(np.mean(list_r2_ns[2]),4), '(', np.round(np.var(list_r2_ns[2]),4), ')')
print('RMSE', np.round(np.mean(list_rmse_ns[2]),4), '(', np.round(np.var(list_rmse_ns[2]),4), ')')
print('Crossentropy', np.round(np.mean(list_crossentropy_ns[2]),4), '(', np.round(np.var(list_crossentropy_ns[2]),4), ')')
print('Cos similarity', np.round(np.mean(list_similarity_ns[2]),4), '(', np.round(np.var(list_similarity_ns[2]),4), ')')
print("---\nDIRICHLET spatial")
print('R2', np.round(np.mean(list_r2_s[2]),4), '(', np.round(np.var(list_r2_s[2]),4), ')')
print('RMSE', np.round(np.mean(list_rmse_s[2]),4), '(', np.round(np.var(list_rmse_s[2]),4), ')')
print('Crossentropy', np.round(np.mean(list_crossentropy_s[2]),4), '(', np.round(np.var(list_crossentropy_s[2]),4), ')')
print('Cos similarity', np.round(np.mean(list_similarity_s[2]),4), '(', np.round(np.var(list_similarity_s[2]),4), ')')

DIRICHLET no spatial
R2 0.9955 ( 0.0 )
RMSE 0.0181 ( 0.0 )
Crossentropy 0.5881 ( 0.0001 )
Cos similarity 0.999 ( 0.0 )
---
DIRICHLET spatial
R2 0.999 ( 0.0 )
RMSE 0.0072 ( 0.0 )
Crossentropy 0.5866 ( 0.0001 )
Cos similarity 0.9999 ( 0.0 )


In [216]:
%%time
list_r2_s, list_rmse_s, list_similarity_s, list_crossentropy_s, list_r2_ns, list_rmse_ns, list_similarity_ns, list_crossentropy_ns = analysis_results(list_solutions_ce_spatial_01, list_solutions_ce_no_spatial_01, rho=0.1)

Wall time: 9.17 s


In [217]:
print("CROSSENTROPY no spatial")
print('R2', np.round(np.mean(list_r2_ns[2]),4), '(', np.round(np.var(list_r2_ns[2]),4), ')')
print('RMSE', np.round(np.mean(list_rmse_ns[2]),4), '(', np.round(np.var(list_rmse_ns[2]),4), ')')
print('Crossentropy', np.round(np.mean(list_crossentropy_ns[2]),4), '(', np.round(np.var(list_crossentropy_ns[2]),4), ')')
print('Cos similarity', np.round(np.mean(list_similarity_ns[2]),4), '(', np.round(np.var(list_similarity_ns[2]),4), ')')
print("---\nCROSSENTROPY spatial")
print('R2', np.round(np.mean(list_r2_s[2]),4), '(', np.round(np.var(list_r2_s[2]),4), ')')
print('RMSE', np.round(np.mean(list_rmse_s[2]),4), '(', np.round(np.var(list_rmse_s[2]),4), ')')
print('Crossentropy', np.round(np.mean(list_crossentropy_s[2]),4), '(', np.round(np.var(list_crossentropy_s[2]),4), ')')
print('Cos similarity', np.round(np.mean(list_similarity_s[2]),4), '(', np.round(np.var(list_similarity_s[2]),4), ')')

CROSSENTROPY no spatial
R2 0.9955 ( 0.0 )
RMSE 0.0179 ( 0.0 )
Crossentropy 0.588 ( 0.0001 )
Cos similarity 0.9991 ( 0.0 )
---
CROSSENTROPY spatial
R2 0.999 ( 0.0 )
RMSE 0.0071 ( 0.0 )
Crossentropy 0.5866 ( 0.0001 )
Cos similarity 0.9999 ( 0.0 )


## rho=0.5

In [34]:
%%time
list_r2_s, list_rmse_s, list_similarity_s, list_crossentropy_s, list_rmse_a_s, list_r2_ns, list_rmse_ns, list_similarity_ns, list_crossentropy_ns, list_rmse_a_ns = analysis_results(list_solutions_spatial_05, list_solutions_no_spatial_05, rho=0.5) 

Wall time: 21.8 s


In [35]:
print("DIRICHLET no spatial")
print('R2', np.round(np.mean(list_r2_ns[2]),4), '(', np.round(np.var(list_r2_ns[2]),4), ')')
print('RMSE', np.round(np.mean(list_rmse_ns[2]),4), '(', np.round(np.var(list_rmse_ns[2]),4), ')')
print('Crossentropy', np.round(np.mean(list_crossentropy_ns[2]),4), '(', np.round(np.var(list_crossentropy_ns[2]),4), ')')
print('Cos similarity', np.round(np.mean(list_similarity_ns[2]),4), '(', np.round(np.var(list_similarity_ns[2]),4), ')')
print('RMSE_A', np.round(np.mean(list_rmse_a_ns[2]),4), '(', np.round(np.var(list_rmse_a_ns[2]),4), ')')
print("---\nDIRICHLET spatial")
print('R2', np.round(np.mean(list_r2_s[2]),4), '(', np.round(np.var(list_r2_s[2]),4), ')')
print('RMSE', np.round(np.mean(list_rmse_s[2]),4), '(', np.round(np.var(list_rmse_s[2]),4), ')')
print('Crossentropy', np.round(np.mean(list_crossentropy_s[2]),4), '(', np.round(np.var(list_crossentropy_s[2]),4), ')')
print('Cos similarity', np.round(np.mean(list_similarity_s[2]),4), '(', np.round(np.var(list_similarity_s[2]),4), ')')
print('RMSE_A', np.round(np.mean(list_rmse_a_s[2]),4), '(', np.round(np.var(list_rmse_a_s[2]),4), ')')

DIRICHLET no spatial
R2 0.8399 ( 0.0001 )
RMSE 0.1138 ( 0.0 )
Crossentropy 0.6303 ( 0.0001 )
Cos similarity 0.9669 ( 0.0 )
RMSE_A 1.2294 ( 0.0035 )
---
DIRICHLET spatial
R2 0.9958 ( 0.0002 )
RMSE 0.0098 ( 0.0001 )
Crossentropy 0.5564 ( 0.0002 )
Cos similarity 0.9994 ( 0.0 )
RMSE_A 0.3738 ( 0.0131 )


In [36]:
list_r2_s, list_rmse_s, list_similarity_s, list_crossentropy_s, list_rmse_a_s, list_r2_ns, list_rmse_ns, list_similarity_ns, list_crossentropy_ns, list_rmse_a_ns = analysis_results(list_solutions_ce_spatial_05, list_solutions_ce_no_spatial_05, rho=0.5)

In [37]:
print("CROSSENTROPY no spatial")
print('R2', np.round(np.mean(list_r2_ns[2]),4), '(', np.round(np.var(list_r2_ns[2]),4), ')')
print('RMSE', np.round(np.mean(list_rmse_ns[2]),4), '(', np.round(np.var(list_rmse_ns[2]),4), ')')
print('Crossentropy', np.round(np.mean(list_crossentropy_ns[2]),4), '(', np.round(np.var(list_crossentropy_ns[2]),4), ')')
print('Cos similarity', np.round(np.mean(list_similarity_ns[2]),4), '(', np.round(np.var(list_similarity_ns[2]),4), ')')
print('RMSE_A', np.round(np.mean(list_rmse_a_ns[2]),4), '(', np.round(np.var(list_rmse_a_ns[2]),4), ')')
print("---\nCROSSENTROPY spatial")
print('R2', np.round(np.mean(list_r2_s[2]),4), '(', np.round(np.var(list_r2_s[2]),4), ')')
print('RMSE', np.round(np.mean(list_rmse_s[2]),4), '(', np.round(np.var(list_rmse_s[2]),4), ')')
print('Crossentropy', np.round(np.mean(list_crossentropy_s[2]),4), '(', np.round(np.var(list_crossentropy_s[2]),4), ')')
print('Cos similarity', np.round(np.mean(list_similarity_s[2]),4), '(', np.round(np.var(list_similarity_s[2]),4), ')')
print('RMSE_A', np.round(np.mean(list_rmse_a_s[2]),4), '(', np.round(np.var(list_rmse_a_s[2]),4), ')')

CROSSENTROPY no spatial
R2 0.865 ( 0.0001 )
RMSE 0.1059 ( 0.0 )
Crossentropy 0.6166 ( 0.0001 )
Cos similarity 0.9686 ( 0.0 )
RMSE_A 0.9236 ( 0.002 )
---
CROSSENTROPY spatial
R2 0.9991 ( 0.0 )
RMSE 0.0072 ( 0.0 )
Crossentropy 0.5553 ( 0.0002 )
Cos similarity 0.9999 ( 0.0 )
RMSE_A 0.3521 ( 0.0011 )


In [218]:
%%time
list_r2_s, list_rmse_s, list_similarity_s, list_crossentropy_s, list_r2_ns, list_rmse_ns, list_similarity_ns, list_crossentropy_ns = analysis_results(list_solutions_spatial_05, list_solutions_no_spatial_05, rho=0.5) 

Wall time: 9.04 s


In [219]:
print("DIRICHLET no spatial")
print('R2', np.round(np.mean(list_r2_ns[2]),4), '(', np.round(np.var(list_r2_ns[2]),4), ')')
print('RMSE', np.round(np.mean(list_rmse_ns[2]),4), '(', np.round(np.var(list_rmse_ns[2]),4), ')')
print('Crossentropy', np.round(np.mean(list_crossentropy_ns[2]),4), '(', np.round(np.var(list_crossentropy_ns[2]),4), ')')
print('Cos similarity', np.round(np.mean(list_similarity_ns[2]),4), '(', np.round(np.var(list_similarity_ns[2]),4), ')')
print("---\nDIRICHLET spatial")
print('R2', np.round(np.mean(list_r2_s[2]),4), '(', np.round(np.var(list_r2_s[2]),4), ')')
print('RMSE', np.round(np.mean(list_rmse_s[2]),4), '(', np.round(np.var(list_rmse_s[2]),4), ')')
print('Crossentropy', np.round(np.mean(list_crossentropy_s[2]),4), '(', np.round(np.var(list_crossentropy_s[2]),4), ')')
print('Cos similarity', np.round(np.mean(list_similarity_s[2]),4), '(', np.round(np.var(list_similarity_s[2]),4), ')')

DIRICHLET no spatial
R2 0.84 ( 0.0001 )
RMSE 0.1138 ( 0.0 )
Crossentropy 0.6303 ( 0.0001 )
Cos similarity 0.9669 ( 0.0 )
---
DIRICHLET spatial
R2 0.9958 ( 0.0002 )
RMSE 0.0098 ( 0.0001 )
Crossentropy 0.5565 ( 0.0002 )
Cos similarity 0.9994 ( 0.0 )


In [220]:
list_r2_s, list_rmse_s, list_similarity_s, list_crossentropy_s, list_r2_ns, list_rmse_ns, list_similarity_ns, list_crossentropy_ns = analysis_results(list_solutions_ce_spatial_05, list_solutions_ce_no_spatial_05, rho=0.5)

In [221]:
print("CROSSENTROPY no spatial")
print('R2', np.round(np.mean(list_r2_ns[2]),4), '(', np.round(np.var(list_r2_ns[2]),4), ')')
print('RMSE', np.round(np.mean(list_rmse_ns[2]),4), '(', np.round(np.var(list_rmse_ns[2]),4), ')')
print('Crossentropy', np.round(np.mean(list_crossentropy_ns[2]),4), '(', np.round(np.var(list_crossentropy_ns[2]),4), ')')
print('Cos similarity', np.round(np.mean(list_similarity_ns[2]),4), '(', np.round(np.var(list_similarity_ns[2]),4), ')')
print("---\nCROSSENTROPY spatial")
print('R2', np.round(np.mean(list_r2_s[2]),4), '(', np.round(np.var(list_r2_s[2]),4), ')')
print('RMSE', np.round(np.mean(list_rmse_s[2]),4), '(', np.round(np.var(list_rmse_s[2]),4), ')')
print('Crossentropy', np.round(np.mean(list_crossentropy_s[2]),4), '(', np.round(np.var(list_crossentropy_s[2]),4), ')')
print('Cos similarity', np.round(np.mean(list_similarity_s[2]),4), '(', np.round(np.var(list_similarity_s[2]),4), ')')

CROSSENTROPY no spatial
R2 0.865 ( 0.0001 )
RMSE 0.1059 ( 0.0 )
Crossentropy 0.6166 ( 0.0001 )
Cos similarity 0.9686 ( 0.0 )
---
CROSSENTROPY spatial
R2 0.9991 ( 0.0 )
RMSE 0.0072 ( 0.0 )
Crossentropy 0.5553 ( 0.0002 )
Cos similarity 0.9999 ( 0.0 )


## rho=0.9

In [29]:
%%time
list_r2_s, list_rmse_s, list_similarity_s, list_crossentropy_s, list_rmse_a_s, list_r2_ns, list_rmse_ns, list_similarity_ns, list_crossentropy_ns, list_rmse_a_ns = analysis_results(list_solutions_spatial_09, list_solutions_no_spatial_09, rho=0.9) 

Wall time: 10.2 s


In [30]:
print("DIRICHLET no spatial")
print('R2', np.round(np.mean(list_r2_ns[2]),4), '(', np.round(np.var(list_r2_ns[2]),4), ')')
print('RMSE', np.round(np.mean(list_rmse_ns[2]),4), '(', np.round(np.var(list_rmse_ns[2]),4), ')')
print('Crossentropy', np.round(np.mean(list_crossentropy_ns[2]),4), '(', np.round(np.var(list_crossentropy_ns[2]),4), ')')
print('Cos similarity', np.round(np.mean(list_similarity_ns[2]),4), '(', np.round(np.var(list_similarity_ns[2]),4), ')')
print('RMSE_A', np.round(np.mean(list_rmse_a_ns[2]),4), '(', np.round(np.var(list_rmse_a_ns[2]),4), ')')

print("---\nDIRICHLET spatial")
print('R2', np.round(np.mean(list_r2_s[2]),4), '(', np.round(np.var(list_r2_s[2]),4), ')')
print('RMSE', np.round(np.mean(list_rmse_s[2]),4), '(', np.round(np.var(list_rmse_s[2]),4), ')')
print('Crossentropy', np.round(np.mean(list_crossentropy_s[2]),4), '(', np.round(np.var(list_crossentropy_s[2]),4), ')')
print('Cos similarity', np.round(np.mean(list_similarity_s[2]),4), '(', np.round(np.var(list_similarity_s[2]),4), ')')
print('RMSE_A', np.round(np.mean(list_rmse_a_s[2]),4), '(', np.round(np.var(list_rmse_a_s[2]),4), ')')


DIRICHLET no spatial
R2 0.1208 ( 0.0063 )
RMSE 0.3245 ( 0.0001 )
Crossentropy 0.8872 ( 0.0008 )
Cos similarity 0.7777 ( 0.0002 )
RMSE_A 4.0079 ( 0.0205 )
---
DIRICHLET spatial
R2 0.8891 ( 0.0093 )
RMSE 0.0871 ( 0.0039 )
Crossentropy 0.4109 ( 0.0057 )
Cos similarity 0.9784 ( 0.0004 )
RMSE_A 2.4673 ( 0.1146 )


In [40]:
list_r2_s, list_rmse_s, list_similarity_s, list_crossentropy_s, list_r2_ns, list_rmse_ns, list_similarity_ns, list_crossentropy_ns = analysis_results(list_solutions_ce_spatial_09, list_solutions_ce_no_spatial_09, rho=0.9) 

ValueError: too many values to unpack (expected 8)

In [ ]:
print("CROSSENTROPY no spatial")
print('R2', np.round(np.mean(list_r2_ns[2]),4), '(', np.round(np.var(list_r2_ns[2]),4), ')')
print('RMSE', np.round(np.mean(list_rmse_ns[2]),4), '(', np.round(np.var(list_rmse_ns[2]),4), ')')
print('Crossentropy', np.round(np.mean(list_crossentropy_ns[2]),4), '(', np.round(np.var(list_crossentropy_ns[2]),4), ')')
print('Cos similarity', np.round(np.mean(list_similarity_ns[2]),4), '(', np.round(np.var(list_similarity_ns[2]),4), ')')
print('RMSE_A', np.round(np.mean(list_rmse_a_ns[2]),4), '(', np.round(np.var(list_rmse_a_ns[2]),4), ')')

print("---\nCROSSENTROPY spatial")
print('R2', np.round(np.mean(list_r2_s[2]),4), '(', np.round(np.var(list_r2_s[2]),4), ')')
print('RMSE', np.round(np.mean(list_rmse_s[2]),4), '(', np.round(np.var(list_rmse_s[2]),4), ')')
print('Crossentropy', np.round(np.mean(list_crossentropy_s[2]),4), '(', np.round(np.var(list_crossentropy_s[2]),4), ')')
print('Cos similarity', np.round(np.mean(list_similarity_s[2]),4), '(', np.round(np.var(list_similarity_s[2]),4), ')')
print('RMSE_A', np.round(np.mean(list_rmse_a_s[2]),4), '(', np.round(np.var(list_rmse_a_s[2]),4), ')')


In [222]:
%%time
list_r2_s2, list_r2_ns2, list_true_mu, list_mu_s, list_mu_ns = details_r2_mu(list_solutions_spatial_09, list_solutions_no_spatial_09, rho=0.9) 

Wall time: 7.07 s


In [31]:
%%time
list_r2_s, list_rmse_s, list_similarity_s, list_crossentropy_s, list_r2_ns, list_rmse_ns, list_similarity_ns, list_crossentropy_ns = analysis_results(list_solutions_spatial_09, list_solutions_no_spatial_09, rho=0.9) 

Wall time: 9.31 s


In [225]:
print("DIRICHLET no spatial")
print('R2', np.round(np.mean(list_r2_ns[2]),4), '(', np.round(np.var(list_r2_ns[2]),4), ')')
print('RMSE', np.round(np.mean(list_rmse_ns[2]),4), '(', np.round(np.var(list_rmse_ns[2]),4), ')')
print('Crossentropy', np.round(np.mean(list_crossentropy_ns[2]),4), '(', np.round(np.var(list_crossentropy_ns[2]),4), ')')
print('Cos similarity', np.round(np.mean(list_similarity_ns[2]),4), '(', np.round(np.var(list_similarity_ns[2]),4), ')')
print("---\nDIRICHLET spatial")
print('R2', np.round(np.mean(list_r2_s[2]),4), '(', np.round(np.var(list_r2_s[2]),4), ')')
print('RMSE', np.round(np.mean(list_rmse_s[2]),4), '(', np.round(np.var(list_rmse_s[2]),4), ')')
print('Crossentropy', np.round(np.mean(list_crossentropy_s[2]),4), '(', np.round(np.var(list_crossentropy_s[2]),4), ')')
print('Cos similarity', np.round(np.mean(list_similarity_s[2]),4), '(', np.round(np.var(list_similarity_s[2]),4), ')')

DIRICHLET no spatial
R2 0.1207 ( 0.0063 )
RMSE 0.3245 ( 0.0001 )
Crossentropy 0.8872 ( 0.0008 )
Cos similarity 0.7777 ( 0.0002 )
---
DIRICHLET spatial
R2 0.8891 ( 0.0093 )
RMSE 0.087 ( 0.0039 )
Crossentropy 0.4108 ( 0.0057 )
Cos similarity 0.9784 ( 0.0004 )


In [32]:
list_r2_s, list_rmse_s, list_similarity_s, list_crossentropy_s, list_rmse_a_s, list_r2_ns, list_rmse_ns, list_similarity_ns, list_crossentropy_ns, list_rmse_a_ns = analysis_results(list_solutions_ce_spatial_09, list_solutions_ce_no_spatial_09, rho=0.9) 

In [34]:
print("CROSSENTROPY no spatial")
print('R2', np.round(np.mean(list_r2_ns[2]),4), '(', np.round(np.var(list_r2_ns[2]),4), ')')
print('RMSE', np.round(np.mean(list_rmse_ns[2]),4), '(', np.round(np.var(list_rmse_ns[2]),4), ')')
print('Crossentropy', np.round(np.mean(list_crossentropy_ns[2]),4), '(', np.round(np.var(list_crossentropy_ns[2]),4), ')')
print('Cos similarity', np.round(np.mean(list_similarity_ns[2]),4), '(', np.round(np.var(list_similarity_ns[2]),4), ')')
print('RMSE_A', np.round(np.mean(list_rmse_a_ns[2]),4), '(', np.round(np.var(list_rmse_a_ns[2]),4), ')')

print("---\nCROSSENTROPY spatial")
print('R2', np.round(np.mean(list_r2_s[2]),4), '(', np.round(np.var(list_r2_s[2]),4), ')')
print('RMSE', np.round(np.mean(list_rmse_s[2]),4), '(', np.round(np.var(list_rmse_s[2]),4), ')')
print('Crossentropy', np.round(np.mean(list_crossentropy_s[2]),4), '(', np.round(np.var(list_crossentropy_s[2]),4), ')')
print('Cos similarity', np.round(np.mean(list_similarity_s[2]),4), '(', np.round(np.var(list_similarity_s[2]),4), ')')
print('RMSE_A', np.round(np.mean(list_rmse_a_s[2]),4), '(', np.round(np.var(list_rmse_a_s[2]),4), ')')


CROSSENTROPY no spatial
R2 0.2382 ( 0.0027 )
RMSE 0.3057 ( 0.0002 )
Crossentropy 0.8307 ( 0.0023 )
Cos similarity 0.792 ( 0.0004 )
RMSE_A 3.7639 ( 0.0213 )
---
CROSSENTROPY spatial
R2 0.9996 ( 0.0 )
RMSE 0.0059 ( 0.0 )
Crossentropy 0.3312 ( 0.0008 )
Cos similarity 0.9999 ( 0.0 )
RMSE_A 2.3476 ( 0.1366 )


In [226]:
list_r2_s, list_rmse_s, list_similarity_s, list_crossentropy_s, list_r2_ns, list_rmse_ns, list_similarity_ns, list_crossentropy_ns = analysis_results(list_solutions_ce_spatial_09, list_solutions_ce_no_spatial_09, rho=0.9) 

In [227]:
print("CROSSENTROPY no spatial")
print('R2', np.round(np.mean(list_r2_ns[2]),4), '(', np.round(np.var(list_r2_ns[2]),4), ')')
print('RMSE', np.round(np.mean(list_rmse_ns[2]),4), '(', np.round(np.var(list_rmse_ns[2]),4), ')')
print('Crossentropy', np.round(np.mean(list_crossentropy_ns[2]),4), '(', np.round(np.var(list_crossentropy_ns[2]),4), ')')
print('Cos similarity', np.round(np.mean(list_similarity_ns[2]),4), '(', np.round(np.var(list_similarity_ns[2]),4), ')')
print("---\nCROSSENTROPY spatial")
print('R2', np.round(np.mean(list_r2_s[2]),4), '(', np.round(np.var(list_r2_s[2]),4), ')')
print('RMSE', np.round(np.mean(list_rmse_s[2]),4), '(', np.round(np.var(list_rmse_s[2]),4), ')')
print('Crossentropy', np.round(np.mean(list_crossentropy_s[2]),4), '(', np.round(np.var(list_crossentropy_s[2]),4), ')')
print('Cos similarity', np.round(np.mean(list_similarity_s[2]),4), '(', np.round(np.var(list_similarity_s[2]),4), ')')

CROSSENTROPY no spatial
R2 0.2382 ( 0.0027 )
RMSE 0.3057 ( 0.0002 )
Crossentropy 0.8307 ( 0.0023 )
Cos similarity 0.792 ( 0.0004 )
---
CROSSENTROPY spatial
R2 0.9996 ( 0.0 )
RMSE 0.0059 ( 0.0 )
Crossentropy 0.3312 ( 0.0008 )
Cos similarity 0.9999 ( 0.0 )
